# snowfall retrieval 
vsnow_hmrr.pro

# The retrieval uses the following:

    - RR:           true rainrate profile from input file
    - rainrate:     retrieved rainrate profile
    - rainrate_a:   a priori guess at rainrate (set to 5.0 mm/hr)
    
    - y_obs:        observed reflectivities
    - y_sim:        simulated refelectivities
    
    - S_a_matrix:   a priori error covariance matrix
    - S_y_matrix:   measurement error covariance matrix
    - S_x_matrix:   retrieval error covariance matrix
    
    - K_matrix:     matrix of Kernel functions
    - A_matrix:     averaging Kernel
    
    - y_contrib:    measurement contribution matrix (Dy Sy Dy^T)
    - a_contrib:    a priori contribution matrix (Da Sa Da^T)
    - LWP_contrib:  LWP contribution matrix (D_LWP sigma_LWP^2 DLWP^T)

In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import pandas as pd
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import createFolder as cF
import save_fig as SF
import math
import scipy.constants as const

import sub24hnum as sub
import read_MRR as pMRR
import calc_date as cd
%matplotlib inline
from decimal import *
getcontext().prec = 7



In [ ]:
date_blue = np.array([1,74,159])/255.    
def plt_refl(ax0, time_MRR, height_MRR, Ze, calday, day, calmon, year):
    levels = np.arange(-10,30,0.2)
    CS = ax0.contourf(time_MRR, height_MRR , Ze, 
                  levels, cmap='jet')
# add colorbar
    cbaxes = fig.add_axes([0.14, 0.1, .75, .02] )   #[left, bottom, width, height] 
    cbar = plt.colorbar(CS, orientation = 'horizontal', cax=cbaxes)
    cbar.ax.set_xlabel('MRR reflectivity [dBz]', fontsize = 22)
    cbar.ax.tick_params(labelsize = 20)

# labels
    times = [0, 3, 6, 9,12, 15, 18, 21, 24]
    ax0.set_xticks(np.arange(0,60*60*25,3*60*60))
    ax0.set_xticklabels(times, fontsize = 20)
    ax0.set_xlabel('time [hours]', fontsize = 22)

    ax0.set_ylabel('height [km]', fontsize = 22)
    ax0.set_ylim(0,3.5)
    ax0.set_yticks(np.arange(0,3500.,500.))
    yl = [0., '' , 1.0, '' , 2., '' , 3.]
    ax0.set_yticklabels(yl, fontsize = 20)
    
    
# textbox
    ax0.text(0.02,0.96, '%s, %s %s %s' %(calday, day, calmon, year), verticalalignment = 'top',  
         horizontalalignment='left',
             transform = ax0.transAxes,
            color =date_blue, fontsize=30,
           bbox={'facecolor':'white','alpha':1., 'pad':10})




In [ ]:
year = '2016'
mon = '12'
day = '23'

In [ ]:
calday, calmon = cd.get_dayname(year,mon, day)

In [ ]:
sfig = 0
fig_dir = '../../Figures/Retrieval/'
cF.createFolder(fig_dir)
form = 'png'


In [ ]:
#### 200m, processed MRR file ########################################
MRR_dir = '../../Data/MRR/processed_MRR/'
fnMRR = netCDF4.Dataset('%s/VMRR_%s%s%s.nc' %(MRR_dir,year,mon,day) ,'r')

time_MRR = fnMRR.variables['time'][:]
height_MRR = fnMRR.variables['height'][:]

Ze = pMRR.read_and_mask(fnMRR,'Ze', np.nan)         # vertical Ze profile for retrieval
Ze.astype(np.float32)
W = pMRR.read_and_mask(fnMRR, 'mean_doppler_velocity', np.nan)    # Doppler data for fallspeed analyses
W = (W) * (-1)

In [ ]:
num_kazr = Ze.shape[0]         # 1440, number of radar profiles
nlayers = Ze.shape[1]          # 14, every 200m
nx = nlayers + nlayers

In [ ]:
#### processed MetNo, temperature file ########################################
temp_dir = '../../Data/sfc_temp/nc'
fnT = netCDF4.Dataset('%s/Haukeli_sfc_temp_%s%s%s.nc' %(temp_dir,year,mon,day),'r')

time_temp = fnT.variables['time'][:].astype(np.float32)        # stime ...surface time
sfc_temp = fnT.variables['sfc_temp'][:].astype(np.float32)    # stemp ...surface temperature

In [ ]:
ntemp = sfc_temp.shape[0]

In [ ]:
#### define values ###########################
# line 83:


In [ ]:
#bad = 0.
# line 112:
#### START RETRIEVAL LOOP ###########################
#for icol2 in range(0,num_kazr):
# line 111:
#    print(icol2)

# line 125:
# surftemp = stemp(icol2)
# vmax = max(vdop(icol2,0:5))
# v4 = max(y_obs)
# if (v4 gt -15.0 and surftemp lt 2.0) then begin      ; Ze mask to save time if no snow
# else begin
# goto, skipend

# line 149:
# constraint_flag = 0              ; Set to 1 to use LWP constraint

# line 160:
# pi = 3.141592653
# comment = ''
# maxiter = 10                     ; Maximum number of iterations
# perturbations = 0.2              ; % value to perturb the retrieved rainrate in calculation of the K-matrix
# delta_Z = 200.                   ; Layer thickness in m

# line 172:
# nprofiles = 1
# nrequencies = 1

# line 199:
# bad_rainrate_flag = 0

# line 275:
# tguess = stemp(icol2)+273
# for ilapse = 0, nlayers-1 do begin
#     t_apriori(ilapse) = tguess-1.*float(ilapse)          ;  ~ moist adiabat 5K/km 1K/200m
# endfor; ilapse

# line 282:
# for iss = 0, nlayers-1 do begin
#    ap_slp(iss)= -(0.03053)*(t_apriori(iss)-273.0)-0.08258         ; a priori log(lambda,ap)
#    ap_n0(iss)=  -(0.07193)* (t_apriori(iss)-273)+2.665            ; a priori log(N0,ap)
#    endfor; iss

# line 303:
# for nkbins=0, nlayers-1 do begin
#    slp(nkbins)=ap_slp(nkbins)
# endfor; nkbins
# for nkbins=nlayers, nx-1 do begin
#    slp(nkbins)=ap_n0(nkbins-nlayers)
# endfor; nkbins

# line 314:
# for i=0, nlayers-1 do begin
#    slpa(i) = ap_slp(i)
# for i=nlayers, nx-1 do begin
#    slpa(i) = ap_n0(i-nlayers)

# line 325:
# for i=0,nlayers-1 do begin
#    Symatrix(i,i) = (2.5)^2       ; 2 dBz
# for i=0, nlayers-1 do begin
#    Samatrix(i,i) = 0.133         ; from Wood et al.
# for i=nlayers, nx-1 do begin
#    Samatrix(i,i) = 0.95

# line 339:
# for iter=0,maxiter-1 do begin
#    print,' '
#    print,'Iteration:',iter+1

# line 345:
# upperturb = slp
# downperturb=slp
# for i= 0 , nx-1 do begin
#    upperturb(i)   = slp(i)*(1.0 + perturbation/100.0)
#    downperturb(i) = slp(i)*(1.0 - perturbation/100.0)

# line 358:
# sub24hnum, upperturb, y_max, iwcpsd, nlayers, delta_z
# sub24hnum, downperturb, y_min, iwcpsd, nlayers, delta_z

# line 363:
# for j=0,nlayers-1 do begin
# Kmatrix(i,j) = (y_max(j)-y_min(j))/(upperturb(i)-downperturb(i))

# line 375:
# sub24hnum, slp, y_sim, iwcpsd, nlayers, delta_z

# line 399:
# if(constraint_flag eq 0) then begin
#     Sxmatrix = invert(invert(Samatrix) + transpose(Kmatrix)##invert(Symatrix)##Kmatrix)
#     slp      = Sxmatrix##(invert(Samatrix)##slpa + transpose(Kmatrix)##invert(Symatrix)##(y_obs - y_sim + Kmatrix##slp))
# endif else begin
#    Sxmatrix = invert(invert(Samatrix) + transpose(Kmatrix)##invert(Symatrix)##Kmatrix + delta_Z^2*transpose(L)##L/sigma_LWP^2)
#    slp      = Sxmatrix##(invert(Samatrix)##slpa + transpose(Kmatrix)##invert(Symatrix)##(y_obs - y_sim + Kmatrix##slp) + delta_Z*transpose(L)*(LWP_obs - LWP_sim)/sigma_LWP^2 + delta_Z^2*transpose(L)##L##slp/sigma_LWP^2)


In [ ]:
h_snow = []
bad = 0.0
## line 112:
#### START RETRIEVAL LOOP ###########################
for icol2 in range(0,num_kazr):
#for icol2 in range(0,1):
    print('icol2',icol2)
#### DEFINE VALUES ###########################
    t_apriori    = np.zeros(shape=nlayers, dtype=np.float32)
    
    slp          = np.zeros(shape=nx,dtype=np.float32)                     # retrieval vector  ??? same as ap_slp, ap_N0
    slpa         = np.zeros(shape=nx,dtype=np.float32)
    
    S_y_matrix   = []                     # measurement error covariance matrix;
    S_a_matrix   = []                     # a priori error covariance matrix;
    
    up_perturb   = np.zeros(shape=nx,dtype=np.float32)
    down_perturb = np.zeros(shape=nx,dtype=np.float32)
    y_max        = np.zeros(shape=nlayers,dtype=np.float32)
    y_min        = np.zeros(shape=nlayers,dtype=np.float32)
    y_sim        = np.zeros(shape=nlayers,dtype=np.float32)
    IWC_psd      = np.zeros(shape=nlayers,dtype=np.float32)
    K_matrix     = np.zeros(shape=(nx,nlayers),dtype=np.float32)
    
    slp_temp     = np.zeros(shape=nx,dtype=np.float32)
    
    S_x_matrix   = np.zeros(shape=(nx,nx),dtype=np.float32)     # retrieval error covariance matrix
##############################################

#### READ IN VALUES ###########################
    y_obs = Ze[icol2, :].astype(np.float32)                # vertical Ze profile for retrieval, observed reflectivities
    yobs = Ze[:,:].astype(np.float32)                      # Ze(time, profiles) for plots
    vdop = W[:,:].astype(np.float32)                       # get Doppler data for fallspeed analyses
##############################################

#### VALID VALUES ZE> -15, SURFACE TEMP < 2 ###########################
## line 125:    
#    surftemp = sfc_temp[icol2]
    vmax = np.nanmax(vdop[icol2,0:6])

#    snow = np.nanmax(y_obs)
    if (np.nanmax(Ze[icol2, :]) > -15. and sfc_temp[icol2] < 2.):      # ; Ze mask to save time if no snow, then go to the next Ze profile
        idx = np.where(np.logical_and(Ze[icol2, :] > -15., sfc_temp[icol2]< 2.))
        h_snow.append(400.)
##### in retrieval without h_snow ##########    
    else:
        h_snow.append(np.nan)
        continue
##############################################



#### NOISE/ERROR PARAMETERS ###########################       
## line 149:     
    constrain_flag    = 0                  # set to 1 to use LWP constraint
#### OTHER INPUT PARAMETERS (shouldn't need to be changed for the project) ###########################       
## line 160:
    pi                = const.pi
    comment           = ''
    max_iter          = 10                  # maximum number of iterations
    perturbation      = 0.2                 # % value to perturb the retrieved rainrate in calculation of the K-matrix
    delta_z           = 200.                # layer thickness in m
## line 172:
    nprofiles         = 1
    nrequencies       = 1
## line 199:
    bad_rainrate_flag = 0
##############################################


#### DEFINE A PRIORI VALUES VIA WOOD ET AL PAPER PAGE 7 ###########################
## line 275:
    tguess = sfc_temp[icol2]+273.
    for ilapse in range(0,nlayers):
        t_apriori[ilapse] = (tguess - 1. * ilapse)               # ~moist adiabat 5K/km 1K/200m
##############################################


#### DEFINE A PRIORI PSD INFO (LOG FORM) THROUGH WOOD TEMPERATURE PARAMETERIZATION ###########################
## line 282:
    ap_slp       = np.zeros(shape = nlayers,dtype=np.float32)
    ap_N0        = np.zeros(shape = nlayers,dtype=np.float32)
    for iss in range(0, nlayers):
        ap_slp[iss] = (-0.03053*(t_apriori[iss]-273.0)-0.08258)          # a priori log(lambda,ap))
        ap_N0[iss]  = ( -0.07193*(t_apriori[iss]-273.0)+2.665 )           # a priori log(N0,ap))
##############################################

#### DEFINE INITIAL GUESS FOR RETRIEVAL VECTOR (PSD INFORMATION) THROUGH A PRIORI GUESS ###########################
## line 303:
    for nkbins in range(0,nlayers):
        slp[nkbins] = (ap_slp[nkbins])
    
    for nkbins in range(nlayers, nx):
        slp[nkbins] = (ap_N0[nkbins-nlayers])
##############################################

#### RE-WRITE APRIORI TERMS (ap_slp, ap_N0) INTO A PRIORI TERM 'slpa' ###########################
#    for i in range(0,nlayers):           # same as before --> rename slp to slpa
 #       slpa.append(ap_slp[i])
  #  for i in range(nlayers, nx):
   #     slpa.append(ap_N0[i-nlayers])
    #print(pd.DataFrame([slp, slpa]))
    slpa = slp
##############################################


#### SET-UP Sy (FORWARD MODEL AND INSTRUMENT UNCERTAINTIES) AND ###########################
#### Sa (A PRIORI) ERROR COVARIANCE MATRICES ###########################
    for i in range(0,nlayers):
        S_y_matrix.append([0] * nlayers)  
    for i in range(0, nx):
        S_a_matrix.append([0] * nx)
    for i in range(0,nlayers):                      
        S_y_matrix[i][i] = 2.5**2                             # 2dBz  
        S_a_matrix[i][i] = 0.133                              # from Wood et al.
    for i in range(nlayers,nx):
        S_a_matrix[i][i] = 0.95
        
    S_y_matrix = (np.asarray(S_y_matrix).astype(np.float32))
    S_a_matrix = (np.asarray(S_a_matrix).astype(np.float32))
##############################################

        

#### NOW BEGIN RETRIEVAL LOOP ###########################
    for iteration in range(0, max_iter):
#    for iteration in range(0, 1):
        print('')
        print('Iteration:', iteration+1)
        
        
#### COMPUTE THE SENSITIVITY MATRIX, K, THROUGH PERTURBING RETRIEVAL VECTOR ###########################
        for i in range(0, nx):
            up_perturb[i]   = ((slp[i]) * (1. + perturbation/100.))
            down_perturb[i] = ((slp[i]) * (1. - perturbation/100.))
##############################################
        

#### CALL FORWARD MODEL FOR PERTURBATIONS ###########################
        for i in range(0, nx):
#        for i in range(0,2):
            print('i',i)
            y_max, IWC_psd = sub.sub24hnum(up_perturb, nlayers, delta_z)
#            print('ymax',y_max)#, 'IWC_psd',IWC_psd)
 #           print('up_perturb', up_perturb)
            y_min, IWC_psd= sub.sub24hnum(down_perturb, nlayers, delta_z)
 #           print('ymin',y_min)#, 'IWC_psd',IWC_psd)
   #         print('down_perturb',down_perturb)
            

# line 363:
            for j in range(0,nlayers):
    #            print('y_diff',y_max[j] - y_min[j])
     #           print('perturp_diff',up_perturb[i]-down_perturb[i])
                K_matrix[i,j] = (y_max[j] - y_min[j])/(up_perturb[i]-down_perturb[i])   # matrix of Kernel functions
          #  print(K_matrix)    
            up_perturb[i]   = slp[i]
            down_perturb[i] = slp[i]  
##############################################

#### CALL FORWARD MODEL FOR Y_SIM ###########################
# line 375:
        y_sim, IWC_psd = sub.sub24hnum(slp, nlayers, delta_z)
##############################################

#### PERFORM RETRIEVAL ###########################
# line 399:
#        if constrain_flag == 0:
 #           S_x_matrix = np.linalg.inv( np.linalg.inv(S_a_matrix) + (np.matmul(np.transpose(K_matrix), np.linalg.inv(S_y_matrix)), K_matrix ))
  #          print(S_x_matrix)
##############################################


In [ ]:
#################################################################################################
######## the following is not in the retrieval just to plot the case  y_obs > -15., surftemp < 2.##################   
list1 = (np.where(np.isnan(h_snow)))
list1 = np.asarray(list1)

list2 = []
list2.append(list1[0,0])
for i in range(0,list1.shape[1]-1):
    if (list1[0,i+1] - list1[0,i] > 1):
        list2.extend([list1[0,i], list1[0,i+1]])
list2 = np.asarray(list2)

idx2 = []
for i in range(0,list2.shape[0]-1,2):
    if (list2[i+1] - list2[i] >=40):
        idx2.extend([list2[i], list2[i+1]])

for k in range(0, np.asarray(idx2).shape[0],2):
    for i in range(idx2[k],idx2[k+1]):
        h_snow[i] = 1

### plot reflectivity
fig = plt.figure(figsize=(20,7))
gs = gridspec.GridSpec(7,1)
ax0 = fig.add_subplot(gs[:6,:])
plt_refl(ax0, time_MRR, height_MRR, np.transpose(Ze), calday, day, calmon, year)



for k in range(0, np.asarray(idx2).shape[0],2):
    dots = ax0.plot(np.asarray(time_MRR)[idx2[k]:idx2[k+1]],np.asarray(h_snow)[idx2[k]:idx2[k+1]],color='purple',
                   linestyle='-',linewidth=5)
    dots2 = ax0.plot(np.asarray(time_MRR)[idx2[k]:idx2[k+1]],2998.+np.asarray(h_snow)[idx2[k]:idx2[k+1]],color='purple',
                   linestyle='-',linewidth=5)
    ax0.axvline(np.asarray(time_MRR)[idx2[k]], color='purple', linestyle='-',linewidth=5)
    ax0.axvline(np.asarray(time_MRR)[idx2[k+1]], color='purple', linestyle='-',linewidth=5)

if sfig == 1:
    fig_name = 'MRR_%s%s%s.%s' %(year,mon,day,form)
    SF.save_figure_landscape(fig_dir, fig_name, form)
else:
    plt.show()
plt.close()
#################################################################################################
